In [1]:
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Model
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_train = (x_train-127.5)/127.5
x_test = (x_test-127.5)/127.5

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(64)

In [5]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(64)

In [6]:
print(train_ds)
print(test_ds)

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>
<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>


In [7]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x): 
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
#@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    #template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print('Epoch {}/{}, Loss: {:.5f}, Accuracy: {:.5f}, Test Loss: {:.5f}, Test Accuracy: {:.5f}'.
            format(
                epoch+1,
                EPOCHS,
                train_loss.result(),
                train_accuracy.result()*100,
                test_loss.result(),
                test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/10, Loss: 0.33588, Accuracy: 89.93667, Test Loss: 0.21352, Test Accuracy: 93.68000
Epoch 2/10, Loss: 0.18106, Accuracy: 94.61167, Test Loss: 0.14592, Test Accuracy: 95.58000
Epoch 3/10, Loss: 0.13398, Accuracy: 95.98333, Test Loss: 0.12421, Test Accuracy: 96.33000
Epoch 4/10, Loss: 0.11040, Accuracy: 96.64333, Test Loss: 0.12326, Test Accuracy: 96.16000
Epoch 5/10, Loss: 0.09862, Accuracy: 97.00166, Test Loss: 0.10790, Test Accuracy: 96.50000
